In [ ]:
print("Start!!")

Start!!


In [ ]:
!pip install -q SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 24.4 MB/s eta 0:00:00


In [ ]:
import os
import speech_recognition as sr
import pandas as pd
import numpy as np

In [ ]:
def transcribe_audio(audio_folder):
    unrecognizable = 0
    transcriptions = []
    for sub_folder in os.listdir(audio_folder):
      sub_folder_path = os.path.join(audio_folder, sub_folder)
      for filename in os.listdir(sub_folder_path):
          if filename.endswith('.wav'):
            file_path = os.path.join(sub_folder_path, filename)
            recognizer = sr.Recognizer()
            with sr.AudioFile(file_path) as source:
                audio_data = recognizer.record(source)
                try:
                    text = recognizer.recognize_google(audio_data)
                except sr.UnknownValueError:
                    print(f"Google Speech Recognition could not understand audio {filename}")
                    text = None
                    unrecognizable+=1
                except sr.RequestError as e:
                    print(f"Could not request results from Google Speech Recognition service; {e}")
                    text = None
            #Append filename and transcription to list
            transcriptions.append({'file_name': filename, 'text': text})
    print("Unrecognizable audios:", unrecognizable)
    return transcriptions

audio_folder = '/content/drive/MyDrive/Medical Speech, Transcription, and Intent/data/recordings'

transcriptions = transcribe_audio(audio_folder)

df_transcriptions = pd.DataFrame(transcriptions)
df_transcriptions

Google Speech Recognition could not understand audio 1249120_44259428_34986999.wav
Google Speech Recognition could not understand audio 1249120_44259428_28102523.wav
Google Speech Recognition could not understand audio 1249120_44259428_56194683.wav
Google Speech Recognition could not understand audio 1249120_44259428_54532216.wav
Google Speech Recognition could not understand audio 1249120_44259428_71308387.wav
Google Speech Recognition could not understand audio 1249120_44263136_34327747.wav
Google Speech Recognition could not understand audio 1249120_44294866_106050928.wav
Google Speech Recognition could not understand audio 1249120_44294866_104430582.wav
Google Speech Recognition could not understand audio 1249120_44294866_21761084.wav
Google Speech Recognition could not understand audio 1249120_44294866_74679338.wav
Google Speech Recognition could not understand audio 1249120_44415714_12774994.wav
Google Speech Recognition could not understand audio 1249120_43898158_10515634.wav
Go

,file_name,text
0,1249120_44246595_105657955.wav,I have a feeling like my whole body is complai...
1,1249120_44246595_107248522.wav,I noticed an important increase in my vision w...
2,1249120_44246595_101823153.wav,I feel pain in my right shoulder after the ten...
3,1249120_44246595_89466082.wav,I often get a stomach ache after I eat I haven...
4,1249120_44246595_92462780.wav,he was discovered to have an open wound
...,...,...
6662,Copy of 1249120_44246595_77258445.wav,I was playing soccer and I twisted my ankle
6663,Copy of 1249120_44246595_76562955.wav,when I go to Stan my head starts swirling
6664,Copy of 1249120_44246595_86895734.wav,surgical wound infections
6665,Copy of Copy of 1249120_44246595_77258445.wav,I was playing soccer and I twisted my ankle


In [ ]:
#Save
df_transcriptions.to_csv('/content/drive/MyDrive/Medical Speech, Transcription, and Intent/data/transcriptions.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Medical Speech, Transcription, and Intent/data/overview-of-recordings.csv')
df.head()

,audio_clipping,audio_clipping:confidence,background_noise_audible,background_noise_audible:confidence,overall_quality_of_the_audio,quiet_speaker,quiet_speaker:confidence,speaker_id,file_download,file_name,phrase,prompt,writer_id
0,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,43453425,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43453425_58166571.wav,When I remember her I feel down,Emotional pain,21665495
1,light_clipping,0.6803,no_noise,0.6803,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...,Hair falling out,44088126
2,no_clipping,1.0000,no_noise,0.6655,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_53187202.wav,there is too much pain when i move my arm,Heart hurts,44292353
3,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,31349958,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...,Infected wound,43755034
4,no_clipping,1.0000,no_noise,1.0000,4.67,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_82524191.wav,My muscles in my lower back are aching,Infected wound,21665495


In [ ]:
# Merge
merged_df = pd.merge(df_transcriptions, df, on='file_name', how='inner')

merged_df = merged_df.dropna(subset=['text'])
merged_df = merged_df[['file_name', 'text', 'phrase', 'prompt']]
merged_df = merged_df.rename(columns={'text': 'converted_phrase', 'prompt': 'label'})

merged_df

,file_name,converted_phrase,phrase,label
0,1249120_44246595_105657955.wav,I have a feeling like my whole body is complai...,I have a feeling like my whole body is complai...,Joint pain
1,1249120_44246595_107248522.wav,I noticed an important increase in my vision w...,I noticed an important decrease in my vision w...,Blurry vision
2,1249120_44246595_101823153.wav,I feel pain in my right shoulder after the ten...,I feel pain in my right shoulder after the ten...,Injury from sports
3,1249120_44246595_89466082.wav,I often get a stomach ache after I eat I haven...,I often get a stomach ache after I eat. I hav...,Stomach ache
4,1249120_44246595_92462780.wav,he was discovered to have an open wound,He was discovered to have an open wound.,Open wound
...,...,...,...,...
6656,1249120_44246595_60134578.wav,I have a painful cramp in my feet,I have a painful cramp in my feet,Foot ache
6657,1249120_44246595_72677345.wav,I have a pain in my trapeze,i have a pain in my trapes,Muscle pain
6658,1249120_44246595_57907447.wav,my shower drain is full of hair every time,My shower drain is full of hair every time.,Hair falling out
6659,1249120_44246595_62158232.wav,my stomach aches when I eat hot food why,"My stomach aches when I eat hot food, why?",Stomach ache


In [ ]:
#Save merged
merged_df.to_csv('/content/drive/MyDrive/Medical Speech, Transcription, and Intent/data/merged_data.csv', index=False)